In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=f78dacdf0ee3c7dfb34488cac99eebe054d95db8491ecde3e1677280ffd7ec58
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Students DF').getOrCreate()

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
df = spark.read.csv("drive/My Drive/Colab Notebooks/students.csv",header=True,inferSchema=True)
df.show()
df.printSchema()

Mounted at /content/drive/
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group B|          bachelor's degree|    standard|                   none|        72|           72|           74|
|female|       group C|               some college|    standard|              completed|        69|           90|           88|
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|
|  male|       group A|         associate's degree|free/reduced|                   none|        47|           57|           44|
|  male|       group C|               some college|    standard|             

In [ ]:
type(df)

pyspark.sql.dataframe.DataFrame

In [ ]:
#1.Number of male and female students

df.select("gender").distinct().count()

df.select("gender").distinct().show()

df.groupBy("gender").count().show()

+------+
|gender|
+------+
|female|
|  male|
+------+

+------+-----+
|gender|count|
+------+-----+
|female|  518|
|  male|  482|
+------+-----+



In [ ]:
#List different 'race/ethnicity
df.select("race/ethnicity").distinct().show()

df.select("race/ethnicity").distinct()\
.orderBy(col("race/ethnicity"), ascending=False).show()

df.select("race/ethnicity").distinct()\
.orderBy(col("race/ethnicity"), ascending=True).show()

df.select("race/ethnicity").distinct().count()

df.groupBy("race/ethnicity").count().show()


+--------------+
|race/ethnicity|
+--------------+
|       group B|
|       group C|
|       group D|
|       group A|
|       group E|
+--------------+

+--------------+
|race/ethnicity|
+--------------+
|       group E|
|       group D|
|       group C|
|       group B|
|       group A|
+--------------+

+--------------+
|race/ethnicity|
+--------------+
|       group A|
|       group B|
|       group C|
|       group D|
|       group E|
+--------------+

+--------------+-----+
|race/ethnicity|count|
+--------------+-----+
|       group B|  190|
|       group C|  319|
|       group D|  262|
|       group A|   89|
|       group E|  140|
+--------------+-----+



In [ ]:
#3.3.What are different 'parental level of education'?

df.select("parental level of education").show()

df.select("parental level of education").distinct().show()

df.select("parental level of education").count()

df.select("parental level of education").distinct().count()

df.groupBy("parental level of education").count().show()

+---------------------------+
|parental level of education|
+---------------------------+
|          bachelor's degree|
|               some college|
|            master's degree|
|         associate's degree|
|               some college|
|         associate's degree|
|               some college|
|               some college|
|                high school|
|                high school|
|         associate's degree|
|         associate's degree|
|                high school|
|               some college|
|            master's degree|
|           some high school|
|                high school|
|           some high school|
|            master's degree|
|         associate's degree|
+---------------------------+
only showing top 20 rows

+---------------------------+
|parental level of education|
+---------------------------+
|           some high school|
|         associate's degree|
|                high school|
|          bachelor's degree|
|            master's degree|
|             

In [ ]:
#4.How many female students scored more than 79 marks in math, whose parental level of education is 'high school'

df.filter(df["math score"] >= 79).orderBy("math score",ascending=True).show()

df.filter((df["gender"]=="female")& (df["math score"] >= 79)).orderBy("math score",ascending=True).show()

df.filter((df["gender"]=="female")& (df["math score"] >= 79)&(df["parental level of education"]=="high school")).orderBy("math score",ascending=True).show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|  male|       group A|           some high school|free/reduced|                   none|        79|           82|           73|
|  male|       group B|           some high school|    standard|              completed|        79|           85|           86|
|female|       group D|          bachelor's degree|    standard|                   none|        79|           89|           89|
|female|       group B|               some college|    standard|                   none|        79|           86|           92|
|female|       group E|         associate's degree|    standard|              completed|        79|     

In [ ]:
#.Check whether average maths score of male or female students are high?
avg1=df.select(avg("math score")).show()

female=df.filter(df["gender"]=="female")\
.select(avg("math score")).show()
male=df.filter(df["gender"]=="male")\
.select(avg("math score")).show()

average = df.groupBy("gender").agg(avg("math score").alias("avg_math_score"))
average.show()

male = average.filter(average["gender"] == "male").collect()[0]["avg_math_score"]
female= average.filter(average["gender"] == "female").collect()[0]["avg_math_score"]
if male > female:
    print("male students is higher.")
elif female > male:
    print(" female students is higher.")


+---------------+
|avg(math score)|
+---------------+
|         66.089|
+---------------+

+------------------+
|   avg(math score)|
+------------------+
|63.633204633204635|
+------------------+

+-----------------+
|  avg(math score)|
+-----------------+
|68.72821576763485|
+-----------------+

+------+------------------+
|gender|    avg_math_score|
+------+------------------+
|female|63.633204633204635|
|  male| 68.72821576763485|
+------+------------------+

male students is higher.


In [ ]:
#6.What is average reading score of male and female students?

average = df.groupBy("gender").agg(avg("reading score").alias("avg_reading_score"))
average.show()

+------+-----------------+
|gender|avg_reading_score|
+------+-----------------+
|female|72.60810810810811|
|  male|65.47302904564316|
+------+-----------------+



In [ ]:
#Whether students score depends upon 'parental level of education'? Justifyyour answer

average= df.groupBy("parental level of education") \
    .agg(avg("math score").alias("avg_math_score"),
         avg("reading score").alias("avg_reading_score"),
         avg("writing score").alias("avg_writing_score"))
average.show()

+---------------------------+------------------+-----------------+-----------------+
|parental level of education|    avg_math_score|avg_reading_score|avg_writing_score|
+---------------------------+------------------+-----------------+-----------------+
|           some high school|63.497206703910614|66.93854748603351|64.88826815642459|
|         associate's degree| 67.88288288288288|70.92792792792793| 69.8963963963964|
|                high school| 62.13775510204081|64.70408163265306|62.44897959183673|
|          bachelor's degree| 69.38983050847457|             73.0|73.38135593220339|
|            master's degree|  69.7457627118644|75.37288135593221|75.67796610169492|
|               some college|  67.1283185840708|69.46017699115045|68.84070796460178|
+---------------------------+------------------+-----------------+-----------------+



In [ ]:
math_score = average.orderBy("avg_math_score", ascending=False).first()
reading_score = average.orderBy("avg_reading_score", ascending=False).first()
writing_score = average.orderBy("avg_writing_score", ascending=False).first()
print("maths",math_score)
print("read",reading_score)
print("writing",writing_score)

maths Row(parental level of education="master's degree", avg_math_score=69.7457627118644, avg_reading_score=75.37288135593221, avg_writing_score=75.67796610169492)
read Row(parental level of education="master's degree", avg_math_score=69.7457627118644, avg_reading_score=75.37288135593221, avg_writing_score=75.67796610169492)
writing Row(parental level of education="master's degree", avg_math_score=69.7457627118644, avg_reading_score=75.37288135593221, avg_writing_score=75.67796610169492)


In [ ]:
#8.List the records where 'test preparation course' is 'none' and scored more than 70 in maths.

df.filter(df["test preparation course"]=="none")\
.where(df["math score"]>=70).count()

df.filter((df["test preparation course"]=="none")&(df["math score"]>=70)).count()

df.filter(df["test preparation course"]=="none")\
.where(df["math score"]>=70).show()

df.filter(df["test preparation course"]=="none")\
.where(df["math score"]>=70)\
.orderBy(df["math score"],ascending=False).show()

df.filter((df["test preparation course"]=="none")& (df["gender"]=="female"))\
.where(df["math score"]>=70)\
.orderBy(df["math score"],ascending=False).show()

df.filter((df["test preparation course"]=="none")& (df["gender"]=="male"))\
.where(df["math score"]>=70)\
.orderBy(df["math score"],ascending=False).show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group B|          bachelor's degree|    standard|                   none|        72|           72|           74|
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|
|  male|       group C|               some college|    standard|                   none|        76|           78|           75|
|female|       group B|         associate's degree|    standard|                   none|        71|           83|           78|
|  male|       group C|                high school|    standard|                   none|        88|     